In [1]:
from __future__ import division

In [2]:
import nltk
import matplotlib as plt
import os
from nltk.tokenize import RegexpTokenizer
import numpy as np
from sklearn.preprocessing import minmax_scale
from IPython.display import Image
from IPython.core.display import HTML 
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from urllib.request import urlopen
import pprint

In [22]:
#School Reading by Grades: Sixth Year

tt1 = "School Readings by Grades: Sixth Year"
url = "http://www.gutenberg.org/files/36864/36864-0.txt"
response = urlopen(url)
txt1 = response.read().decode('utf8')
type(txt1)

str

In [23]:
#School Reading By Grades: Fifth Year

tt2 = "School Reading By Grades: Fifth Year"
url = "http://www.gutenberg.org/files/51000/51000-0.txt"
response = urlopen(url)
txt2 = response.read().decode('utf8')
type(txt2)

str

In [24]:
#Little Dramas for Primary Grades

tt3 = "Little Dramas for Primary Grades"
url = "http://www.gutenberg.org/files/53245/53245-0.txt"
response = urlopen(url)
txt3 = response.read().decode('utf8')
type(txt3)

str

In [25]:
#Morning in the West

tt4 = "Morning in the West"
url = "http://www.gutenberg.org/cache/epub/63153/pg63153.txt"
response = urlopen(url)
txt4 = response.read().decode('utf8')
type(txt4)

str

In [28]:
#Men Who Have Made the Empire

tt5 = "Men Who Have Made the Empire"
url = "https://www.gutenberg.org/files/63148/63148-0.txt"
response = urlopen(url)
txt5 = response.read().decode('utf8')
type(txt5)

str

In [29]:
#Save our titles for later
titles = [tt1, tt2, tt3, tt4, tt5]
titles

['School Readings by Grades: Sixth Year',
 'School Reading By Grades: Fifth Year',
 'Little Dramas for Primary Grades',
 'Morning in the West',
 'Men Who Have Made the Empire']

In [8]:
#Double check to make sure the texts were imported correclty
len(txt5)

411686

In [9]:
#Text sample to ensure we imported correctly
txt3[:75]

'\ufeffThe Project Gutenberg EBook of Little Dramas for Primary Grades, by \r\nAda '

### Tokenize each text so we can find the vocab size of each text

In [10]:
from nltk import word_tokenize
#nltk.download('punkt')

In [11]:
#Combine all texts into one list so we can iterate through
text_list = [txt1, txt2, txt3, txt4, txt5]

In [12]:
#Create a list with the tokenized texts
tok_textlist = []
for i in text_list:
    tokens = word_tokenize(i)
    i = nltk.Text(tokens)
    tok_textlist.append(i)

In [13]:
#Ensure we have the correct data type
for i in tok_textlist:
    print(type(i))
    print(len(i))

<class 'nltk.text.Text'>
72533
<class 'nltk.text.Text'>
55249
<class 'nltk.text.Text'>
22137
<class 'nltk.text.Text'>
10355
<class 'nltk.text.Text'>
79868


# Question 1.)
>1.	In Python, create a method for scoring the vocabulary size of a text, and normalize the score from 0 to 1. It does not matter what method you use for normalization as long as you explain it in a short paragraph. (Various methods will be discussed in the live session.)

### Vocabulary Size Funciton

In [14]:
#Function to score vocab size
#Note: Function takes in a list of texts
def vocab_size(x):
    vsize = []
    normed = []
    
    for i in x:
        vsize.append(len(set(i)))
        
    for j in vsize:
        normed.append((j - min(vsize)) / (max(vsize) - min(vsize)))
    return (normed)

In [15]:
vocab_size(tok_textlist)

[1.0, 0.8460052435490548, 0.056713122671450254, 0.0, 0.915275286325376]

The method I used to normalize the data was to do the actual formula of:
> (Score  - Min_Score) / (Max_Score – Min_Score)

This will bring the scores to a value between 0 and 1.  This will allow us to use other metrics without having to be concerned about the varying scales of the metrics and actually compare apples to apples.   If we did not use a normalization technique, we wouldn't be able to relaly use vocab size, long word score, and lexical diversity in the same sentence since the scale for all three are so different. 


# Question 2.)
>2.	After consulting section 3.2 in chapter 1 of Bird-Klein, create a method for scoring the long-word vocabulary size of a text, and likewise normalize (and explain) the scoring as in step 1 above.

## Long Word Score Function from Chapter 1 Section 3.2

In [16]:
def long_words(x):
    long_list = []
    nlong_list = []
    temp_cnt = 0
    for i in x:
        for j in i: 
            if len(j)>=15:
                temp_cnt += 1
        long_list.append(temp_cnt)
        temp_cnt = 0
    
    #Normalization of scores     
    for j in long_list:
        nlong_list.append((j - min(long_list)) / (max(long_list) - min(long_list)))
    return nlong_list
                

In [17]:
long_words(tok_textlist)

[0.12844036697247707, 0.009174311926605505, 0.22018348623853212, 0.0, 1.0]

Here we normalize the scores so they can be compatible with the scores from question 1.  If we were not to do this, the scales of the two would be drastically different, and combining the two would not be comparing apples to apples.  For example, long-word vocab size of a text is somewhere between 10 and 60.  The vocab size score from question 1 is at least one, more so two levels of magnitude larger.  Once we normalize them, they are on the same playing field. 

# Question 3.)
>3.	Now create a “text difficulty score” by combining the lexical diversity score from homework 1, and your normalized score of vocabulary size and long-word vocabulary size, in equal weighting. Explain what you see when this score is applied to same graded texts you used in homework 1.

##### Lexical Diversity Funciton from HW 1

In [18]:
def lex_diversity(x):
    lex_list = []
    
    for i in x:
        lex_list.append(len(set(i)) / len(i))
    return lex_list

In [19]:
lex_diversity(tok_textlist)

[0.13745467580273807,
 0.1602562942315698,
 0.1415729321949677,
 0.26296475132786096,
 0.11714328642259729]

## Text Difficulty Function Combining All Three

In [71]:
def text_difficulty(x):
    #run our three functions against text list
    vs = np.array(vocab_size(x))
    lw = np.array(long_words(x))
    ld = np.array(lex_diversity(x))
    
    #convert into matrix where the text is the column value
    score_matrix = np.concatenate(np.array([vs, lw, ld]), axis=0)
    
    #reshape the matrix and sum the 'columns' 
    score_matrix.shape = (3, len(x))
    final_scores = score_matrix.sum(axis=0)

    #combine final_scores and titles into sorted dictionary to easily identify the most and least difficult
    tmp = {j:i for i,j in sorted(zip(final_scores, titles), reverse=True)}
    
    return tmp

In [72]:
text_difficulty(tok_textlist)

{'Men Who Have Made the Empire': 2.0324185727479733,
 'School Readings by Grades: Sixth Year': 1.2658950427752154,
 'School Reading By Grades: Fifth Year': 1.01543584970723,
 'Little Dramas for Primary Grades': 0.41846954110495005,
 'Morning in the West': 0.26296475132786096}

Now that we have taken more than one factor into account, the text difficulty scores of the graded texts seem to align with my expectations—the texts would get more difficult as the grades increased.  The first three texts I used in this assignment were the three texts from homework one and decreased in grade from 1-3 (sixth, fifth, primary).  The corresponding text difficulty scores were 1.266, 1.012, and 0.418.  This seems much more reasonable than using lexical diversity or vocabulary size of a text alone.  